# Join
Copyright (c) Microsoft Corporation. All rights reserved.<br>
Licensed under the MIT License.<br>

In DataPrep you can easily join two dataflows.

In [1]:
!pip install azureml

In [2]:
import azureml.dataprep as dprep

First let's get the left side of the data into a shape that is ready for the join.

In [3]:
# get the first dataflow and derive desired key column
dataflow_l = dprep.read_csv(path='https://dpreptestfiles.blob.core.windows.net/testfiles/BostonWeather.csv')
dataflow_l = dataflow_l.derive_column_by_example(source_columns='DATE', new_column_name='date_timerange',
                                                 example_data=[('11/11/2015 0:54', 'Nov 11, 2015 | 12AM-2AM'),
                                                              ('2/1/2015 0:54', 'Feb 1, 2015 | 12AM-2AM'),
                                                              ('1/29/2015 20:54', 'Jan 29, 2015 | 8PM-10PM')])
dataflow_l = dataflow_l.drop_columns(['DATE'])

# convert types and summarize data
dataflow_l = dataflow_l.set_column_types(type_conversions={'HOURLYDRYBULBTEMPF': dprep.TypeConverter(dprep.FieldType.DECIMAL)})
dataflow_l = dataflow_l.filter(expression=dprep.f_not(dprep.col('HOURLYDRYBULBTEMPF').is_error()))
dataflow_l = dataflow_l.summarize(group_by_columns=['date_timerange'],summary_columns=[dprep.SummaryColumnsValue('HOURLYDRYBULBTEMPF', dprep.api.engineapi.typedefinitions.SummaryFunction.MEAN, 'HOURLYDRYBULBTEMPF_Mean')] )

# cache the result so the steps above are not executed every time we pull on the data
import os
from pathlib import Path
cache_dir = str(Path(os.getcwd(), 'dataflow-cache'))
dataflow_l.cache(directory_path=cache_dir)
dataflow_l.head(10)

ExecutionError: Cannot write cache. Please check if the specified cache folder exists.

Now let's prepare the data for the right side of the join.

In [ ]:
# get the second dataflow and desired key column
dataflow_r = dprep.read_csv(path='https://dpreptestfiles.blob.core.windows.net/bike-share/*-hubway-tripdata.csv')
dataflow_r = dataflow_r.keep_columns(['starttime', 'start station id'])
dataflow_r = dataflow_r.derive_column_by_example(source_columns='starttime', new_column_name='l_date_timerange',
                                                 example_data=[('2015-01-01 00:21:44', 'Jan 1, 2015 | 12AM-2AM')])
dataflow_r = dataflow_r.drop_columns('starttime')

# cache the results
dataflow_r.cache(directory_path=cache_dir)
dataflow_r.head(10)

There are three ways one can join two dataflows in DataPrep:
1. Create `JoinBuilder` object for interactive join configuration.
2. Call ```join()``` on one of the dataflows and pass in the other along with all other arguments.
3. Call ```Dataflow.join()``` method and pass in two dataflows along with all other arguments.

We will explore the builder object as it simplifies the determination of correct arguments. 

In [ ]:
# construct a builder for joining dataflow_l with dataflow_r
join_builder = dataflow_l.builders.join(right_dataflow=dataflow_r, left_column_prefix='l', right_column_prefix='r')

join_builder

As you can see, so far the builder has no propeties set except default values.
From here you could set each of the options and preview its effect on the join result or use DataPrep to determine some of them.

Let's start with determining appropriate column prefixes for left and right side of the join and lists of columns that would not conflict and therefore don't need to be prefixed.  

In [ ]:
join_builder.detect_column_info()
join_builder

You can see that DataPrep has performed a pull on both dataflows to determine the column names in them. Given that `dataflow_r` already had a column starting with `l_` new prefix got generated which would not collide with any column names that are already present.
Additionally columns in each dataflow that won't conflict during join would remain unprefixed.
This apprach to column naming is crucial for join robustness to schema changes in the data. Let's say that at some time in future the data consumed by left dataflow will also have `l_date_timerange` column in it.
Configured as above the join will still run as expected and the new column will be prefixed with `l2_` ensuring that ig column `l_date_timerange` was consumed by some other future transformation it remains unaffected.

Note: `KEY_generated` is appended to both lists and is reserved for Dataprep use in case Autojoin is performed.

### Autojoin
Autojoin is a Dataprep feature that determines suitable join arguments given data on both sides. In some cases Autojoin can even derive a key column from a number of available columns in the data.
Here is how you can use Autojoin:

In [ ]:
# generate join suggestions
join_builder.generate_suggested_join()

# list generated suggestions
join_builder.list_join_suggestions()

Now lets select first suggestion and preview the result of the join.

In [ ]:
# apply first suggestion
join_builder.apply_suggestion(0)

join_builder.preview(10)

Everything looks just as we would expect, so it is time to get our new joined dataflow.

In [ ]:
dataflow_autojoined = join_builder.to_dataflow().drop_columns(['l_date_timerange'])

### Joining two dataflows without pulling the data

In don't want to pull on data and know what join should look like you can always you join method on Dataflow 

In [ ]:
dataflow_joined = dprep.Dataflow.join(left_dataflow=dataflow_l,
                                      right_dataflow=dataflow_r,
                                      join_key_pairs=[('date_timerange', 'l_date_timerange')],
                                      left_column_prefix='l2_',
                                      right_column_prefix='r_')


In [ ]:
dataflow_joined.head(10)

In [ ]:
dataflow_joined = dataflow_joined.filter(expression=dprep.col('r_start station id') == '67')
df = dataflow_joined.to_pandas_dataframe()
df